#  Multi-Objective Evolutionary Optimization: Final Assignment

## Overview

In this notebook, we apply many-objective evolutionary optimization to explore **non-dominated policy sets** for dike reinforcement and spatial interventions within the Overijssel region. Our aim is to identify robust strategies that balance the interests of multiple stakeholders while respecting specific physical and institutional constraints.

## Scope of the Analysis

- **Focus region**: The optimization is focused on two dike rings:
  - **Deventer** (`A.4`)
  - **Grossel** (`A.5`)
  
- **Constraints applied**:
  - **Dike height increase** for both Deventer and Grossel is constrained between **5 and 10 dm**.
  - **Room for the River (RfR) projects** are **activated** only for those of strategic interest to **Rijkswaterstaat (RWS)**:
    - Dike rings: `A.1` (Doesburg), `A.3` (Zupthen), and `A.4` (Grossel)

- **Decision space**:
  - We search over **policy levers**, which include dike reinforcements and RfR implementations.
  - All other interventions are treated as optional and will be explored by the algorithm.

- **Evaluation criteria**:
  - Objectives reflect the priorities of the **Province of Overijssel**, such as minimizing flood risk and economic damages, while preserving social and ecological outcomes.

## Purpose

This analysis uses the `optimize()` functionality from the `ema_workbench`, relying on an $\epsilon$-NSGAII algorithm to:
- Discover a set of **Pareto-optimal policies** under complex trade-offs.
- Support negotiation and coordination between regional actors (e.g., RWS, local municipalities, and the province).
- Identify **robust** strategies that perform well under a wide range of uncertain future scenarios.

## Key Features

- Integration of **realistic policy constraints** from stakeholders.
- Use of $\epsilon$-archiving and **parallel coordinate plots** to visualize trade-offs.
- Inclusion of **convergence metrics** and optional seed analysis to assess solution quality.


In [1]:
import pandas as pd
import networkx as nx
import os
import copy
import numpy as np
from ema_workbench.examples.sd_boostedtrees_flu import experiments, outcomes
from fontTools.misc.bezierTools import epsilon

# Ensure archive directory exists
archive_dir = "./archives"
os.makedirs(archive_dir, exist_ok=True)

/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/precupada/decision_making_assignments/final assignment EPA141/model_scripts/data/1000 flu cases with policies.tar.gz'

In [22]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [23]:
from ema_workbench import (
    Policy,
    ema_logging,
    MultiprocessingEvaluator,
    save_results, 
    load_results,
    Samplers,
    IntegerParameter
)
from problem_formulation import get_model_for_problem_formulation



In [24]:
ema_logging.log_to_stderr(ema_logging.INFO)


<Logger EMA (DEBUG)>


## Overview

This notebook runs the optimization algorithm (ε-NSGAII) to generate a diverse set of non-dominated policy alternatives, and saves the resulting input–output data for further analysis. 

While the MOEA algorithm is executed here via the `optimize()` function, the full analysis of convergence behavior, stakeholder trade-offs, and Pareto front exploration is performed in a follow-up notebook.
### Key Elements

- **Problem Formulation 3 (PF 3)** is used, which provides disaggregated scalar outcomes by dike ring, allowing policy evaluation at the regional level.
- A custom function `MOEA_experiments(...)` is used to:
  - Load the correct problem setup
  - Apply constraints for stakeholder-specific preferences
  - Run ε-NSGAII optimization using the EMA Workbench

### Policy Constraints

The following policy constraints reflect both national and provincial stakeholder preferences:

- **Room for the River projects** are turned ON for dike rings `A.1`, `A.3`, and `A.4` to represent the interests of **Rijkswaterstaat (RWS)**.
- **Dike increases** for Deventer (`A.3`) and Grossel (`A.5`) are limited to **5–10 dm** to reflect preferences of the **Province of Overijssel**.

### Epsilon Configuration

Instead of manually specifying ε-values, they are **automatically generated** based on the range of outcomes from prior exploratory analysis. This improves:
- Adaptability to different problem formulations
- Efficiency and interpretability of the Pareto front

### Output

The optimization results and convergence logs are saved for further analysis, visualization, and stakeholder engagement.



In [25]:
def initialize_model(problem_formulation_id, active_rfr_dikes=None, constrained_dikes=None):
    from problem_formulation import get_model_for_problem_formulation
    dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation_id)

    # Apply RfR forcing
    if active_rfr_dikes:
        new_levers = []
        for lev in dike_model.levers:
            if "RfR" in lev.name and any(dike in lev.name for dike in active_rfr_dikes):
                new_levers.append(IntegerParameter(lev.name, 1, 1))  # Force ON
            else:
                new_levers.append(lev)
        dike_model.levers = new_levers

    # Apply constraints on dike increases
    if constrained_dikes:
        new_levers = []
        for lev in dike_model.levers:
            replaced = False
            for dike, (low, high) in constrained_dikes.items():
                if dike in lev.name and "DikeIncrease" in lev.name:
                    new_levers.append(IntegerParameter(lev.name, low, high))
                    replaced = True
                    break
            if not replaced:
                new_levers.append(lev)
        dike_model.levers = new_levers

    return dike_model

In [26]:
def build_convergence_metrics(dike_model, archive_filename, enable_logging=True):
    from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
    metrics = [EpsilonProgress()]

    if enable_logging:
        archive_dir = "../experimental data"
        os.makedirs(archive_dir, exist_ok=True)
        logger = ArchiveLogger(
            archive_dir,
            [l.name for l in dike_model.levers],
            [o.name for o in dike_model.outcomes],
            base_filename=archive_filename
        )
        metrics.insert(0, logger)
    
    return metrics

In [27]:
def run_optimization(dike_model, nfe, epsilons, convergence_metrics):
    from ema_workbench import MultiprocessingEvaluator
    import time

    print(f"Starting optimization with {nfe} evaluations...")
    
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results, convergence = evaluator.optimize(
            nfe=nfe,
            searchover="levers",
            epsilons=epsilons,
            convergence=convergence_metrics
        )
    
    print(f"Optimization completed.")
    return results, convergence

## Optimization Configuration

To run the MOEA effectively and efficiently, several key parameters were chosen based on best practices and the characteristics of the problem formulation:

### Number of Function Evaluations (`nfe = 20,000`)
We use 20,000 function evaluations to allow the algorithm enough opportunity to explore the high-dimensional decision space and converge toward a diverse, high-quality Pareto front. Literature on many-objective optimization suggests 10k–50k evaluations as a practical range for balancing search depth and runtime.

### Epsilon Thresholds (`epsilons`)
Instead of manually setting epsilon values, we compute them automatically as **5% of each outcome’s range**, based on results from a prior exploratory run. This:
- Ensures outcome-specific granularity
- Prevents overfitting to numerical noise
- Produces a well-distributed and manageable set of non-dominated policies

The formula used is:

\[
\varepsilon_i = 0.05 \times (\max_i - \min_i)
\]

for each outcome \(i\), where \(\max_i\) and \(\min_i\) are calculated from the exploratory `.tar.gz` archive.


In [28]:
def generate_epsilons_from_results(outcomes, fraction=0.05):
    """
    Generate epsilon values based on a fraction of the outcome range.

    Args:
        outcomes (dict): Dictionary of outcomes as returned by `load_results(...)`
        fraction (float): Fraction of the range to use for epsilon (e.g., 0.05 for 5%)

    Returns:
        List of epsilon values, in the order of outcomes.keys()
    """
    import numpy as np

    epsilons = []
    for name, values in outcomes.items():
        values = np.asarray(values)
        min_val, max_val = values.min(), values.max()
        range_val = max_val - min_val

        if range_val < 1e-5:
            epsilon = 1e-5  # avoid zero epsilon
        else:
            epsilon = range_val * fraction

        epsilons.append(epsilon)

    return epsilons

In [19]:
model = initialize_model(
    problem_formulation_id=3,
    active_rfr_dikes=["A.1", "A.3", "A.4"],
    constrained_dikes={"A.3": (5, 10), "A.5": (5, 10)}
)

In [38]:
convergence_metrics = build_convergence_metrics(model, ".tar.gz", enable_logging=False)
# Load exploratory results to compute epsilons
experiments_pf3, outcomes_pf3 = load_results("../experimental data/pf_3_exploratory_runs_levers_as_factors.tar.gz")
epsilon_pf3 = generate_epsilons_from_results(outcomes, fraction=0.05)

[MainProcess/INFO] results loaded successfully from /Users/precupada/decision_making_assignments/final assignment EPA141/experimental data/pf_3_exploratory_runs_levers_as_factors.tar.gz


In [39]:
results_moea, convergence = run_optimization(
    dike_model=model,
    nfe=200,
    epsilons=epsilon_pf3,  
    convergence_metrics=convergence_metrics
)

Starting optimization with 200 evaluations...


/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed -

KeyboardInterrupt: 